In [8]:
# This is a regression problem, not calssification
# Smile generator link
# https://www.cheminfo.org/flavor/malaria/Utilities/SMILES_generator___checker/index.html
# Install deepchem.
%%capture
!pip install deepchem

In [9]:
import deepchem as dc

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [11]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train, valid, test = datasets

In [12]:
# Checking the shape of the data
print(train.X.shape) #There are 902 molecules

(902,)


In [14]:
print(train.y.shape) #The y is the solubility

(902, 1)


In [15]:
#We also have the weight variable
print(train.w.shape)

(902, 1)


In [16]:
# Checking the shape in test dataset
print(test.X.shape)

(113,)


Building the model

In [20]:
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', graph_conv_layers=[128,128], dropout=0.20)
#If we increase the dropout the test_score can increase too

In [21]:
model.fit(train, nb_epoch=100) # The result is the average loss
#By increasing the number of layers the average loss can be less

0.08614117622375489

In [22]:
#Evaluation of the model (creation of the metric)
metrics = dc.metrics.Metric(dc.metrics.pearson_r2_score)
train_scores = model.evaluate(train, [metrics], transformers)
print(train_scores)

test_score= model.evaluate(test, [metrics], transformers)
print(test_score)

{'pearson_r2_score': 0.9404402201562939}
{'pearson_r2_score': 0.6850547678889533}


In [27]:
# Imagine you are designing drugs and want to try their solubilities, how to check that with these model?
smiles = ['Nc1cccc(O)c1', 'CC1CCCC(C)C1']
# We obtained the SMILES representation of a molecule from the SMILE generator

In [28]:
from rdkit import Chem

In [29]:
# First, convert the SMILE string into a mole to be put into the model
mol = [Chem.MolFromSmiles(molecules) for molecules in smiles]
mol

In [30]:
#Featurization of the new molecules
featurizer = dc.feat.ConvMolFeaturizer()
featurized_mol = featurizer.featurize(mol) #Creation of the array and featurization of our mol object

In [31]:
#SOLUBILITY PREDICTION
prediction = model.predict_on_batch(featurized_mol)
print(prediction)

[[ 1.7165319]
 [-0.6107986]]
